Una vez hemos analizado el conjunto de datos que tenemos de las transacciones (ver analisisInicial.py y analisis.html) recogidas durante un mes, destacamos los siguientes resultados:
- Tenemos como entrada (x) 200.000 datos de transacciones (big data).
- No hay transacciones duplicadas ni inválidas.
- Tenemos como salida (y), una clasificación binaria (0,1): fraude o no (columna Is_fraud).
- Tenemos un total de 10088 transacciones etiquetadas comno fraudes (5% de los datos, lo que podría estar desbalanceado).
- Al estar utilizando un dataset ya existente, el etiquetado realizado es correcto y no necesita de un trabajo de etiquetado adicional.


Una vez realizado este análisis, comenzamos con el preprocesamiento de los datos, con el objetivo de establecer un "Data Pipeline". A continuación, se detalla el Data Pipeline definido para preparar los datos antes de comenzar con el modelado ML.

# 1. Limpieza de datos
Vamos a eliminar aquellas variables que no son relevantes ni predictivas:
- **Variables con valores únicos: Customer_ID, Transaction_ID, Merchant_ID, Transaction_Currency.**
- **Variables no predictivas: Customer_Name, State, City, Bank_Branch, Transaction_Date, Transaction_Location, Customer_Contact, Transaction_Description, Customer_Email.** Los datos de las personas no nos servirá predecir si una transacción es fraude por el nombre de la persona. Las transacciones fraudulentas ya han sido denominadas como tal y esa persona no debería volver a poder realizar transacciones. La fecha no es relevante, ya que nuevas transacciones se realizan en nuevas fechas. Las localizaciones de las transacciones actualmente son datos de un solo país, por lo que de momento lo mejor será obviar estas variables debido a la complejidad que puede suponer para nuestro modelo si luego se introducen nuevos paises. Además, al ser muchas ciudades y estados, minimizaremos el riesgo de sobreajuste a pesar de perder información del contexto geográfico.


In [31]:
import pandas as pd

In [32]:
# Cargamos nuestro conjunto de datos original
df = pd.read_csv('data/raw_data.csv')

df.columns

Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email'],
      dtype='object')

In [33]:
# Eliminamos las columnas que no necesitamos
columns_to_drop = [
    "Customer_ID", "Transaction_ID", "Merchant_ID", "Transaction_Currency",  # Variables con valores únicos
    "Customer_Name", "State", "City", "Bank_Branch", "Transaction_Date", "Transaction_Location", 
    "Customer_Contact", "Transaction_Description", "Customer_Email"  # Variables no predictivas
]

df = df.drop(columns=columns_to_drop)

df.columns

Index(['Gender', 'Age', 'Account_Type', 'Transaction_Time',
       'Transaction_Amount', 'Transaction_Type', 'Merchant_Category',
       'Account_Balance', 'Transaction_Device', 'Device_Type', 'Is_Fraud'],
      dtype='object')

# 2. Categorización de variables
Vamos a agrupar variables en categorías, con el objetivo de reducir la complejidad de variables con muchos valores distintos:
- **Transaction_Time: toma valores continuos de la hora de la transacción. Vamos a agrupar las horas en Mañana, tarde y noche.**
- **Transaction_Device: tiene 20 categorías distintas. Vamos a agrupar las categráis comunes (>5%), y las poco representadas (<5%) en 'Other Devices'.**

In [34]:
def categorize_time(time):
    if time >= pd.to_datetime("06:00:00").time() and time < pd.to_datetime("14:00:00").time():
        return "Mañana"
    elif time >= pd.to_datetime("14:00:00").time() and time < pd.to_datetime("21:00:00").time():
        return "Tarde"
    else:
        return "Noche"
    
print(df["Transaction_Time"].value_counts())

df['Transaction_Time'] = pd.to_datetime(df['Transaction_Time'], format='%H:%M:%S').dt.time
    
df["Transaction_Time"] = df["Transaction_Time"].apply(categorize_time)

print(df["Transaction_Time"].value_counts())

Transaction_Time
07:30:31    11
03:29:22    11
13:02:29    11
21:12:43    10
10:50:14    10
            ..
09:15:13     1
09:18:13     1
02:00:43     1
19:38:04     1
17:50:24     1
Name: count, Length: 77856, dtype: int64
Transaction_Time
Noche     74942
Mañana    66701
Tarde     58357
Name: count, dtype: int64


In [35]:
def group_transaction_device(device):
    common_devices = ["Self-service Banking Machine", "ATM", "ATM Booth Kiosk"]
    return device if device in common_devices else "Other Devices"

df["Transaction_Device"] = df["Transaction_Device"].apply(group_transaction_device)

print(df["Transaction_Device"].value_counts())

Transaction_Device
Other Devices                   135944
Self-service Banking Machine     21707
ATM                              21200
ATM Booth Kiosk                  21149
Name: count, dtype: int64


In [36]:
# Columnas actuales del conjunto de datos
print(df.columns)

Index(['Gender', 'Age', 'Account_Type', 'Transaction_Time',
       'Transaction_Amount', 'Transaction_Type', 'Merchant_Category',
       'Account_Balance', 'Transaction_Device', 'Device_Type', 'Is_Fraud'],
      dtype='object')


# 3. Codificación de variables
En el siguiente paso, vamos a tratar algunas de las variables de nuestro conjunto de datos:
- **One-Hot encoding (Codificación de variables categóricas a numéricas): Gender, Account_Type, Transaction_Time, Transaction_Type, Merchant_Category, Transaction_Device, Device_Type.** Vamos a convertir estas columnas a valores numéricos, creando una variable para cada categoría, de forma que tome valores entre 0 y 1. Esto lo hacemos para aquellas variables con pocas categorías.

In [37]:
df = pd.get_dummies(df , columns = ["Gender", "Account_Type", "Transaction_Time" ,"Transaction_Type", "Merchant_Category", "Transaction_Device" ,"Device_Type"], drop_first=True)

In [38]:
print(df.columns)

Index(['Age', 'Transaction_Amount', 'Account_Balance', 'Is_Fraud',
       'Gender_Male', 'Account_Type_Checking', 'Account_Type_Savings',
       'Transaction_Time_Noche', 'Transaction_Time_Tarde',
       'Transaction_Type_Credit', 'Transaction_Type_Debit',
       'Transaction_Type_Transfer', 'Transaction_Type_Withdrawal',
       'Merchant_Category_Electronics', 'Merchant_Category_Entertainment',
       'Merchant_Category_Groceries', 'Merchant_Category_Health',
       'Merchant_Category_Restaurant', 'Transaction_Device_ATM Booth Kiosk',
       'Transaction_Device_Other Devices',
       'Transaction_Device_Self-service Banking Machine',
       'Device_Type_Desktop', 'Device_Type_Mobile', 'Device_Type_POS'],
      dtype='object')


In [39]:
df.head()

,Age,Transaction_Amount,Account_Balance,Is_Fraud,Gender_Male,Account_Type_Checking,Account_Type_Savings,Transaction_Time_Noche,Transaction_Time_Tarde,Transaction_Type_Credit,...,Merchant_Category_Entertainment,Merchant_Category_Groceries,Merchant_Category_Health,Merchant_Category_Restaurant,Transaction_Device_ATM Booth Kiosk,Transaction_Device_Other Devices,Transaction_Device_Self-service Banking Machine,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS
0,60,32415.45,74557.27,0,True,False,True,False,True,False,...,False,False,False,True,False,True,False,False,False,True
1,51,43622.60,74622.66,0,False,False,False,False,True,False,...,False,False,False,True,False,True,False,True,False,False
2,20,63062.56,66817.99,0,True,False,True,True,False,False,...,False,True,False,False,False,False,False,True,False,False
3,57,14000.72,58177.08,0,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False
4,43,18335.16,16108.56,0,False,False,True,False,True,False,...,True,False,False,False,False,True,False,False,True,False


In [40]:
# Nos aseguramos de que tenemos 0 y 1, no booleanos.
# Definir las columnas booleanas que necesitas convertir
boolean_columns = [
    'Gender_Male', 'Account_Type_Checking', 'Account_Type_Savings',
    'Transaction_Time_Noche', 'Transaction_Time_Tarde',
    'Transaction_Type_Credit', 'Transaction_Type_Debit',
    'Transaction_Type_Transfer', 'Transaction_Type_Withdrawal',
    'Merchant_Category_Electronics', 'Merchant_Category_Entertainment',
    'Merchant_Category_Groceries', 'Merchant_Category_Health',
    'Merchant_Category_Restaurant', 'Transaction_Device_ATM Booth Kiosk',
    'Transaction_Device_Other Devices', 'Transaction_Device_Self-service Banking Machine',
    'Device_Type_Desktop', 'Device_Type_Mobile', 'Device_Type_POS'
]

df[boolean_columns] = df[boolean_columns].astype(int)


In [41]:
#Colocamos la varaible is fraud al final
df = df[[col for col in df if col != 'Is_Fraud'] + ['Is_Fraud']]

In [42]:
df.head()

,Age,Transaction_Amount,Account_Balance,Gender_Male,Account_Type_Checking,Account_Type_Savings,Transaction_Time_Noche,Transaction_Time_Tarde,Transaction_Type_Credit,Transaction_Type_Debit,...,Merchant_Category_Groceries,Merchant_Category_Health,Merchant_Category_Restaurant,Transaction_Device_ATM Booth Kiosk,Transaction_Device_Other Devices,Transaction_Device_Self-service Banking Machine,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS,Is_Fraud
0,60,32415.45,74557.27,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
1,51,43622.60,74622.66,0,0,0,0,1,0,0,...,0,0,1,0,1,0,1,0,0,0
2,20,63062.56,66817.99,1,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,57,14000.72,58177.08,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
4,43,18335.16,16108.56,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0


# 4. Normalización y escalado
A continuación, vamos a aplicar un escalado de los datos para las variables numéricas (Age, Transaction_Amount, Account_Balance). Esto asegura que todas las características contribuyan de manera equitativa al modelo. En particular, vamos a aplicar un escalado robusto para además tratar los posibes outliers. Se va a utilizar la mediana y el rango intercuartílico para reducir la influencia de valores extremos, proporcionando una transformación más robusta.

In [43]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

numerical_columns = ['Age', 'Transaction_Amount', 'Account_Balance']

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [44]:
df.head()

,Age,Transaction_Amount,Account_Balance,Gender_Male,Account_Type_Checking,Account_Type_Savings,Transaction_Time_Noche,Transaction_Time_Tarde,Transaction_Type_Credit,Transaction_Type_Debit,...,Merchant_Category_Groceries,Merchant_Category_Health,Merchant_Category_Restaurant,Transaction_Device_ATM Booth Kiosk,Transaction_Device_Other Devices,Transaction_Device_Self-service Banking Machine,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS,Is_Fraud
0,0.615385,-0.345448,0.467980,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
1,0.269231,-0.118873,0.469359,0,0,0,0,1,0,0,...,0,0,1,0,1,0,1,0,0,0
2,-0.923077,0.274145,0.304722,1,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0.500000,-0.717739,0.122445,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
4,-0.038462,-0.630109,-0.764978,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0


Finalmente, vamos a almacenar el nuevo dataset, y obtener un análisis del nuevo conjunto de datos tras el preprocesamiento

In [45]:
df.to_csv('data/clean_data.csv', index=False)

In [46]:
import pandas as pd
from ydata_profiling import ProfileReport

df = df = pd.read_csv('data/clean_data.csv')
profile = ProfileReport(df, title='Pandas Profiling Report')

profile.to_file("analisis_clean_data.html")

c:\Users\javi2\Desktop\MUII\DISIA\Proyecto\git\Bank-Fraud-Detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 166.75it/s]


Con el conjunto de datos limpio y preprocesado, hemos realizado varias acciones clave para asegurar que las variables sean adecuadas para el análisis y la construcción de modelos predictivos. Las principales conclusiones son las siguientes:
- **Eliminación de variables no relevantes ni predictivas:** Hemos identificado y eliminado aquellas variables que no aportan valor predictivo al modelo, como aquellas relacionadas con datos personales o irrelevantes para la predicción del fraude. Esto ayuda a reducir la complejidad del modelo y mejorar su rendimiento.
- **Agrupación de categorías de variables:** Para variables como las relacionadas con el tiempo de la transacción, el tipo de dispositivo, hemos agrupado las categorías en categorías más significativas y manejables (por ejemplo, categorizando el tiempo en "mañana", "tarde" y "noche"). Esta agrupación permite simplificar el modelo y reducir la dimensionalidad de los datos sin perder información crucial.
- **Codificación de variables categóricas a formato numérico (0 y 1):** Hemos transformado las variables categóricas en variables numéricas utilizando One-Hot Encoding. Esto convierte las categorías en columnas binarias (0 o 1), lo que permite que los modelos de Machine Learning las procesen de manera eficiente, sin perder la información categórica.
- **Escalado robusto de las variables numéricas:** Como parte de la normalización de los datos, hemos aplicado un escalado robusto a las variables numéricas. Este tipo de escalado es resistente a los valores atípicos (outliers), ya que utiliza la mediana y el rango intercuartílico en lugar de la media y la desviación estándar. Esto asegura que los modelos no se vean influenciados por los outliers y que todas las variables numéricas estén en la misma escala, lo que mejora la convergencia y rendimiento del modelo.

